In [ ]:
import json
import pandas as pd
with open("outputs/buckets_07-29_charsim17.json","r") as file:
  buckets = json.load(file)
bucket_map={}
for name in buckets:
  for action in buckets[name]:
    bucket_map[action]=name
bucket_names = list(buckets.keys())
bucket_names_inv={name:i for i,name in enumerate(bucket_names)}
datasets = [f"data/US/{term*2+2009-222}-{term*2+2010-222}_{term}th_Congress/csv/history.csv" for term in range(111,120)]
history_df_by_term=[pd.read_csv(ds) for ds in datasets]
for i,df in enumerate(history_df_by_term):
  df["term"]=i+111
history_df=pd.concat(history_df_by_term)
# with open("./outputs/bucket_names_output.jsonl", "r") as file:
#   for line in file:
#     if line.strip():  # Skip empty lines
#       response = json.loads(line)
#       llm_buckets.append(response["response"]["body"]["output"][0]["content"][0]["text"])
      
# history_df["llm_bucket"] = history_df["bucket"].map(dict(zip(bucket_names, llm_buckets)))

# Add next bill_id column to compare
history_df["bucket"]=history_df["action"].apply(lambda action:bucket_map[action])
bills={bill_id:group for (bill_id,group) in history_df.groupby("bill_id")}
bill_ids = list(bills.keys())


In [ ]:
import numpy as np

alpha=2/3
data=[]
MIN_TERM=min(history_df["term"])
MAX_TERM=max(history_df["term"])
N_TERMS=MAX_TERM-MIN_TERM+1

bill_id=history_df.iloc[0]["bill_id"]
predecessors_vec=np.zeros(len(bucket_names))
predecessor_vec=np.zeros(len(bucket_names))
one_hot_curr = np.zeros(len(bucket_names))
inputs=[]
outputs=[]
for i,row in history_df.iterrows():
  one_hot_prev = np.copy(one_hot_curr)
  if row["bill_id"]!=bill_id:
    bill_id=row["bill_id"]
    one_hot_prev=np.zeros(len(bucket_names))
    predecessors_vec=np.zeros(len(bucket_names))
    predecessor_vec=np.zeros(len(bucket_names))
  one_hot_curr = np.zeros(len(bucket_names))
  one_hot_curr[bucket_names_inv[row.bucket]]=1
  predecessor_vec=(1-alpha)*one_hot_prev+alpha*predecessor_vec
  predecessors_vec[bucket_names_inv[row.bucket]]+=1
  one_hot_term=np.zeros(N_TERMS)
  one_hot_term[row.term-MIN_TERM]=1
  input_vec = np.concat([predecessor_vec,predecessors_vec,one_hot_term])
  data.append({"predecessor":predecessor_vec,"predecessors":predecessors_vec,"term":one_hot_term,"output":one_hot_curr})
# inputs=np.stack(inputs)
# outputs=np.stack(outputs)
# np.savetxt("outputs/vectors/07-29_charsim17/input",inputs[0])
# np.savetxt("outputs/vectors/07-29_charsim17/output",outputs)

In [ ]:

import os
import shutil

shutil.rmtree("outputs/vectors/07-29_charsim17")
# os.rmdir("outputs/vectors/07-29_charsim17")
os.mkdir("outputs/vectors/07-29_charsim17")
BATCH_SIZE=100000
data_batches = np.array_split(data,list(range(BATCH_SIZE,len(data),BATCH_SIZE)))
for batch_i, batch in enumerate(data_batches):
  with open(f"outputs/vectors/07-29_charsim17/data{batch_i}","wb") as file:
    np.save(file,batch)